<a href="https://colab.research.google.com/github/nngu0123/ADS2002/blob/main/NN_Regression_MPG_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression using Neural Networks

In the previous notebooks on neural networks we have considered classification problems. Regression problems, where the output is a continuous value, can also be handled using neural networks. To demonstrate this we will use the classic [Auto MPG Dataset](https://archive.ics.uci.edu/ml/datasets/auto+mpg) to build a model to predict the fuel efficiency of 1970s and early 1980s automobiles. This datasets describes fuel efficiency of many cars from that period. This description includes attributes such as: engine cylinders, engine displacement, horsepower, and weight.

We will show how linear regression can be implemented using neural networks, and then consider some simple nonlinear regression models.

## Contents

* Imports
* The Auto MPG Dataset
* Linear Regression
* Nonlinear Regression
* Model Performance
* Exercises

## Imports

First, import the standard libraries. If you are running this on Google Colab, and `seaborn` cannot be found, then uncomment the following cell.

In [ ]:
# !pip install -q seaborn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

We import `tensorflow` and `keras`, the package `layers` for setting up sequential models and a function for normalizing data.

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


## The Auto MPG dataset


We first download and import the dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/) using pandas. The names can be found in the file [auto-mpg.names](https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.names).

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

df = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [ ]:
df.sample(5)

First we check for missing values, which are denoted by `?` in the file, and which the import statement has converted to NaN.

In [ ]:
df.isna().sum()

Drop those rows to keep this simple.

In [ ]:
df.dropna(inplace=True)

The `"Origin"` column is categorical, not numeric. So we first use a dictionary to convert it to categorical data and then one-hot encode with `pd.get_dummies`.

In [ ]:
df['Origin'] = df['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
df = pd.get_dummies(df, columns=['Origin'], prefix='', prefix_sep='')
df.sample(5)

We can view the statistics, to see the range of each variable.

In [ ]:
df.describe()

We can investigate the correlation between variables. The fuel efficiency (MPG) is highly correlated with the variables `Cylinders`, `Displacement`, `Horsepower` and `Weight`, and the correlation between these variables is very high.

In [ ]:
corrs = df.corr() # calculate the correlation table
# as this is a symmetric table,
# set up a mask so that we only plot values below the main diagonal
mask = np.triu(np.ones_like(corrs, dtype=bool))
f, ax = plt.subplots(figsize=(10, 8)) # initialise the plots and axes
# plot the correlations as a seaborn heatmap, with a colourbar
sns.heatmap(corrs, mask=mask, center=0, annot=True, square=True, linewidths=.5)
# do some fiddling so that the top and bottom are not obscured
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5);

`sns.pairplot()` can be used to view the relationship between different features. What we now want to do is create a model which determines the fuel efficiency (MPG) as a function of the other features. It appears that fuel efficiency is approximately inversely proportional to the other variables.

In [ ]:
sns.pairplot(df[['MPG', 'Cylinders', 'Horsepower', 'Displacement', 'Weight']],
             diag_kind='kde', height=2)

Since we want to predict `MPG`, this is our label, and the other variables are features. We can separate the data into the label and features, and then split both sets into testing and training sets.

In [ ]:
from sklearn.model_selection import train_test_split

features = df.drop(['MPG'], axis=1)
labels = df['MPG']

X_train, X_test, y_train, y_test = train_test_split(features, labels,
                                                    test_size=0.2,
                                                    random_state=42)

In the table of statistics it is apparent that the values of the features are widely distributed.

In [ ]:
X_train.describe().loc[['mean', 'std']]

As with classification with neural networks, it is best practice to normalize features that use different scales and ranges. This ensures that techniques such as regularization, which we will consider later, can be applied uniformly.

There is no advantage to normalizing the one-hot features, it is done here for simplicity.

Previously we have used the `mean` and `std` to normalize the variables. Here we will use `keras` `preprocessing.Normalization` layer to build the preprocessing into the model.

The first step is to create the layer. `axis=-1` states to apply the normalization to the last dimension.

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)

Then `.adapt()` it to the data, which calculates the mean and variance, and stores them in the layer.

In [ ]:
normalizer.adapt(np.array(X_train))

In [ ]:
print(normalizer.mean.numpy())

When the layer is called it returns the input data, with each feature independently normalized.

In [ ]:
firstrow = np.array(X_train[:1])

print('First example:', firstrow)
print('Normalized:', normalizer(firstrow).numpy())

## Linear regression



Before building a nonlinear neural network model, we will build a model which implements linear regression. This corresponds to a single perceptron with a continuous output.

First we create a convenience function to build and compile a simple sequential neural network. This is essentially the same as the models we have previously considered for classification, except the last layer is a `Dense` layer, with a single continuous output, and the loss function is the mean absolute error.

For this models we can specify different inputs using `norm`, the number of hidden layers and the neurons in each of these layers, and the learning rate used by SGD.

In [ ]:
def build_model_regress(norm, n_hidden=1, n_neurons=30,
                            learning_rate=0.01):
  """Build and compile a simple sequential model with n_hidden
  layers and n_neurons in each layer."""

  model = keras.models.Sequential()
  model.add(norm)
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons, activation="relu"))
  model.add(keras.layers.Dense(1))

  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(loss='mean_absolute_error', optimizer=optimizer)

  return model

We start by creating a linear regression model which takes the input `Horsepower` and aims to predict `MPG`.

For our default model we first need to create the horsepower `Normalization` layer. This just corresponds to creating an array with the `Horsepower` from the training set, initializing the normalization routine so that the input shape is a single vector (one feature) and then adapting the normalizer to calculate the mean and standard deviation of `Horsepower`.

In [ ]:
horsepower = np.array(X_train['Horsepower'])

horsepower_normalizer = preprocessing.Normalization(input_shape=[1,], axis=None)
horsepower_normalizer.adapt(horsepower)

This layer then can be used as the model input. Since this is linear regression, we want no hidden layers and can set the number of neurons to 0. We can then output a summary of the model.

In [ ]:
horsepower_model = build_model_regress(horsepower_normalizer, n_hidden=0,
                                 n_neurons=0, learning_rate=0.03)

horsepower_model.summary()

Now the model is configured, we use `Model.fit()` to train the model. Here we use 80% of the data for training and 20% for validation. The evolution of the metrics for the model are stored in `history`.

In [ ]:
%%time
history = horsepower_model.fit(
    X_train['Horsepower'], y_train,
    epochs=200,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

We can create a simple function for plotting the history of the model.

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 20])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

For this model the training loss and the validation loss decreases steadily, with the training loss always being less than the validation loss, which is to be expected.

In [ ]:
plot_loss(history)

We evaluate the results and store them in a structure for comparison with the other models.

In [ ]:
test_results = {}

test_results['lin_horsepower_model'] = horsepower_model.evaluate(
    X_test['Horsepower'],
    y_test, verbose=0)

Since this is a single variable linear regression the output corresponds to a linear relationship, and we can compare the model predictions against the actual value.

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = horsepower_model.predict(x)

We define another convenience function for comparing the predictions.

In [ ]:
def plot_horsepower(x, y):
  plt.scatter(X_train['Horsepower'], y_train, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

The predictions are reasonable for mid-range horsepower, but fail at the upper and lower limits.

In [ ]:
plot_horsepower(x,y)

To implement multi-dimensional linear regression, we now just need to input the normalization layer which was defined earlier for the whole data set. Now the input shape corresponds to nine features.

In [ ]:
linear_model = build_model_regress(normalizer, n_hidden=0, n_neurons=0,
                                 learning_rate=0.03)

linear_model.summary()

We can now `fit` the model using the full training dataset.

In [ ]:
%%time
history = linear_model.fit(
    X_train, y_train,
    epochs=200,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Plotting the history demonstrates that using all the inputs achieves a much lower training and validation error than the `horsepower` model.

In [ ]:
plot_loss(history)

We collect the results of the test set, for later comparison.

In [ ]:
test_results['lin_model'] = linear_model.evaluate(
    X_test, y_test, verbose=0)

## Nonlinear regression

The previous section implemented linear models for single and multiple inputs.

This section implements single-input and multiple-input Neural Network models. The code is essentially the same except the model is expanded to include hidden  nonlinear layers.

These models will contain a few more layers than the linear model:

* The normalization layer.
* Two hidden, nonlinear, `Dense` layers using the `relu` nonlinearity.
* A linear single-output layer.


We start with a model for the single input "Horsepower". The only difference is the number of hidden layers, and the number of neurons in these layers. However, there are now significantly more trainable parameters.

In [ ]:
nn_horsepower_model = build_model_regress(horsepower_normalizer, n_hidden=2,
                                     n_neurons=64, learning_rate=0.03)

nn_horsepower_model.summary()

Training of the model is the same as before.

In [ ]:
%%time
history = nn_horsepower_model.fit(
    X_train['Horsepower'], y_train,
    validation_split=0.2,
    verbose=0, epochs=200)

This model is slightly more accurate than the linear-horsepower model, but the initial convergence is much more rapid.

In [ ]:
plot_loss(history)

Plotting the predictions as a function of `Horsepower`, we now see this model takes advantage of the nonlinearity provided by the hidden layers.

In [ ]:
x = tf.linspace(0.0, 250, 251)
y = nn_horsepower_model.predict(x)

plot_horsepower(x, y)

We collect the results of the test set, for later comparison.

In [ ]:
test_results['nn_horsepower_model'] = nn_horsepower_model.evaluate(
    X_test['Horsepower'], y_test,
    verbose=0)

This process can be repeated using all the inputs, which slightly improves the performance on the validation dataset.

In [ ]:
nn_model = build_model_regress(normalizer, n_hidden=2, n_neurons=64,
                          learning_rate=0.03)
nn_model.summary()

In [ ]:
%%time
history = nn_model.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=0, epochs=200)

In [ ]:
plot_loss(history)

Again we collect the results on the test set.

In [ ]:
test_results['nn_model'] = nn_model.evaluate(X_test, y_test, verbose=0)

## Performance

Now that all the models are trained we can compare the performance. Not surprisingly, as the complexity of the model increases the absolute error of the model decrease. This suggests that the final model does not have excessive overfitting.

In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

Finally, we can use `predict` to compare the predicted values for the testing set against the actual values.

In [ ]:
test_predictions = nn_model.predict(X_test).flatten()

plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims,lims);

This suggests the model predicts the fuel efficiency reasonably well, so we can save it for later use.


In [ ]:
nn_model.save('nn_model.hd5');

## Exercises

For these exercises we will investigate regularization techniques to cope with overfitting for the full nonlinear model. There are two-standard techniques for dealing with overfitting.

The first of these is to use L2 (Ridge) or L1 (Lasso) Regularization on each layer. These add a penalty term to the objective function which si proportional to the square or absolute value of the weights. The objective is to reduce the number of non-zero weights in the system. This is analogous to what was previously considered with linear and logistic regression.

The second method is to use dropout layers. In a dropout layer, during training a randomly chosen percentage of the nodes in each layer are ignored at each iteration. This reduces the sensitivity of the network to the training set and hence creates a more robust model.

The function below generalizes the one created earlier to include L2 Regularization and Dropout layers.

In [ ]:
def build_model_reg(norm, n_hidden=1, n_neurons=30,
                            learning_rate=0.01,
                  reg_factor=0, drop_rate=0):
  """Set up and compile a simple sequential model with n_hidden
  layers and n_neurons in each layer, including L2 regularization
  and drop-out layers."""

  model = keras.models.Sequential()
  model.add(norm)
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,
                    kernel_regularizer=keras.regularizers.l2(reg_factor),
                    activation="relu"))
    model.add(keras.layers.Dropout(drop_rate))
  model.add(keras.layers.Dense(1,
                        kernel_regularizer=keras.regularizers.l2(reg_factor)))

  optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
  model.compile(loss='mean_absolute_error', optimizer=optimizer)

  return model

### Exercise 1 (4 marks)

Create a neural network with the same parameters as `nn_model`, but with a regularization factor of 0.001. Train the model, plot the history and store the model results in `test_results`.

In [ ]:
reg_model = build_model_reg(normalizer, n_hidden=2, n_neurons=64,
                          learning_rate=0.03, reg_factor=0.001)
reg_model.summary()

In [ ]:
%%time
history = reg_model.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=0, epochs=200)

In [ ]:
plot_loss(history)

In [ ]:
 test_results['reg_model'] = reg_model.evaluate(X_test, y_test, verbose=0)

### Exercise 2 (4 marks)

Repeat Exercise 1, but this time set the regularization factor to 0 and the dropout rate to 0.2.

In [ ]:
drop_model = build_model_reg(normalizer, n_hidden=2, n_neurons=64,
                          learning_rate=0.03, drop_rate=0.2)
reg_model.summary()

In [ ]:
%%time
history = drop_model.fit(
    X_train, y_train,
    validation_split=0.2,
    verbose=0, epochs=200)

In [ ]:
plot_loss(history)

In [ ]:
test_results['drop_model'] = drop_model.evaluate(X_test, y_test, verbose=0)

### Exercise 3 (2 marks)

Print the comparison of all six models that have been created.

In [ ]:
 pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T